# ECSE 415 Assignment 1
## Author1: Bohan Wang
## Student ID1: 261023725
## Author2: Peter Zhang
## Student ID2: 26102xxxx
## Comments:
### + Library used in this assignment:
#### `numpy version: 1.26.4`
#### `scikit-image version: 0.23.2`
#### `matplotlib version: 3.8.5`


### + Run system setup for install the library
### + Please find path for the images in the `Images path` section


## Citation
```
@misc{ecse-415-a-4-24-f,
    author = {Kevin Kong},
    title = {ECSE415 Image Segmentation},
    year = {2024},
    howpublished = {\url{https://kaggle.com/competitions/ecse-415-a-4-24-f}},
    note = {Kaggle}
}
